In [50]:
import google.generativeai as genai
import os
import numpy as np
from dotenv import load_dotenv 
import json
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

True

In [51]:
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

model_name = os.getenv("MODEL_NAME", "gemini-1.5-flash") 

In [52]:
model = genai.GenerativeModel(model_name)

In [53]:
def get_chatbot_response(messages):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "parts": message["parts"]})
    response = model.generate_content(messages)  
    
    return response

In [54]:
messages = [{'role':'user','parts':"What's the capital of Italy?"}]
response = get_chatbot_response(messages)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Rome\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.003409839700907469
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 9,
        "candidates_token_count": 2,
        "total_token_count": 11
      }
    }),
)


In [12]:
system_prompt = """
            You are a helpful AI assistant for a coffee shop application which serves drinks and pastries.
            Your task is to determine whether the user is asking something relevant to the coffee shop or not.
            The user is allowed to:
            1. Ask questions about the coffee shop, like location, working hours, menue items and coffee shop related questions.
            2. Ask questions about menue items, they can ask for ingredients in an item and more details about the item.
            3. Make an order.
            4. ASk about recommendations of what to buy.

            The user is NOT allowed to:
            1. Ask questions about anything else other than our coffee shop.
            2. Ask questions about the staff or how to make a certain menue item.

            Your output should be in a structured json format like so. each key is a string and each value is a string. Make sure to follow the format exactly:
            {
            "chain of thought": go over each of the points above and make see if the message lies under this point or not. Then you write some your thoughts about what point is this input relevant to.
            "decision": "allowed" or "not allowed". Pick one of those. and only write the word.
            "message": leave the message empty if it's allowed, otherwise write "Sorry, I can't help with that. Can I help you with your order?"
            }
            """

prompt = input("User: ")
messages.append({"role": "user", "parts": prompt})


input_messages = [{"role": "model", "parts": system_prompt}] + messages

In [55]:
system_prompt =  """
            You are a helpful AI assistant for a coffee shop application.
            Your task is to determine what agent should handle the user input. You have 3 agents to choose from:
            1. details_agent: This agent is responsible for answering questions about the coffee shop, like location, delivery places, working hours, details about menue items. Or listing items in the menu items. Or by asking what we have.
            2. order_taking_agent: This agent is responsible for taking orders from the user. It's responsible to have a conversation with the user about the order untill it's complete.
            3. recommendation_agent: This agent is responsible for giving recommendations to the user about what to buy. If the user asks for a recommendation, this agent should be used.

            Your output should be in a structured json format like so. each key is a string and each value is a string. Make sure to follow the format exactly:
            {
            "chain of thought": go over each of the agents above and write some your thoughts about what agent is this input relevant to.
            "decision": "details_agent" or "order_taking_agent" or "recommendation_agent". Pick one of those. and only write the word.
            "message": leave the message empty.
            }
            """
prompt = input("User: ")
messages.append({"role": "user", "parts": prompt})


input_messages = [{"role": "model", "parts": system_prompt}] + messages   

In [56]:
chatbot_output = get_chatbot_response(input_messages).text
chatbot_output

'```json\n{\n  "chain of thought": "The user is asking a question that is unrelated to the coffee shop\'s offerings, location, hours, or menu.  None of the agents are equipped to answer this question. However, since it\'s not a question about ordering or a request for recommendations, the details agent is the least inappropriate choice.",\n  "decision": "details_agent",\n  "message": ""\n}\n```\n'

In [57]:
def postprocess(output):
        cleaned_output = "\n".join(output.splitlines()[1:]).strip('`')
        output = json.loads(cleaned_output)

        dict_output = {
            "role": "assistant",
            "parts": output['message'],
            "memory": {"agent":"classification_agent",
                       "classification_decision": output['decision']
                      }
        }
        return dict_output

In [58]:
output = postprocess(chatbot_output)
output

{'role': 'assistant',
 'parts': '',
 'memory': {'agent': 'classification_agent',
  'classification_decision': 'details_agent'}}

In [59]:
output['memory']['classification_decision']

'details_agent'

In [14]:
def postprocess(output):
        cleaned_output = "\n".join(output.splitlines()[1:]).strip('`')
        output = json.loads(cleaned_output)

        dict_output = {
            "role": "assistant",
            "parts": output['message'],
            "memory": {"agent":"guard_agent",
                       "guard_decision": output['decision']
                      }
        }
        return dict_output

In [15]:
output = postprocess(chatbot_output)
output

{'role': 'assistant',
 'parts': "Sorry, I can't help with that. Can I help you with your order?",
 'memory': {'agent': 'guard_agent', 'guard_decision': 'not allowed'}}

In [ ]:
if output["memory"]["guard_decision"] == "not allowed":
    print("not allowed")
else:
    print("alowed")


okay


In [ ]:
def get_chatbot_response(system_prompt, user_messages):
    model = genai.GenerativeModel(model_name)
    
    chat = model.start_chat(history=[{"role": "model", "parts": [{"text": system_prompt}]}])
    
    for message in user_messages[-3:]:  
        user_message = {"role": "user", "parts": [{"text": message}]}
        response = chat.send_message(user_message)
        

    return chat.history[-1].parts[0].text  

In [95]:
user_messages = [
    "In one sentence, explain how a computer works to a young child.",
    "Okay, how about a more detailed explanation to a high schooler?"
]

response = get_chatbot_response(user_messages)
response


'A computer works by executing a series of instructions, called a program, written in a language it understands.  These instructions manipulate data stored in its memory and on storage devices, performing calculations, manipulating images and text, and interacting with external devices like keyboards, screens, and the internet, all based on the logic defined by the program.\n'

In [84]:
user_prompt = ["what is ramen"]
get_chatbot_response(user_prompt)

'Ramen is a Japanese noodle soup.  It consists of:\n\n* **Broth:**  This is the base of the dish and can be made from a variety of ingredients, most commonly pork, chicken, beef, seafood, or combinations thereof.  Often it includes dashi (a stock made from kombu kelp and bonito flakes), soy sauce, and mirin (sweet rice wine).  The broth is what gives ramen its distinct flavor.\n\n* **Noodles:**  These are typically wheat noodles, but other types can sometimes be used. They are usually thin and slightly alkaline, giving them a characteristic chewiness.\n\n* **Toppings:**  This is where the immense variety of ramen comes in. Common toppings include:\n    * Chashu (braised pork belly)\n    * Soft-boiled eggs (often marinated in soy sauce and mirin)\n    * Nori (dried seaweed sheets)\n    * Menma (fermented bamboo shoots)\n    * Scallions (green onions)\n    * Bean sprouts\n    * Corn\n    * Mushrooms\n    * Spiced oil\n\n\nWhile instant ramen is a popular and widely available quick meal, 

In [18]:
def get_chatbot_response(messages):
    response = model.generate_content(messages)  
    
    return response

In [19]:
system_prompt = "You are a helpful assistant."  
messages = [{'text': system_prompt}]  
messages.append({'text': "What's the capital of Italy?"}) 

response = get_chatbot_response(messages)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The capital of Italy is Rome.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.00046353484503924847
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 15,
        "candidates_token_count": 8,
        "total_token_count": 23
      }
    }),
)


### Three techniques of prompt engineering:

### 1) structured output 

In [10]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    country: the country that you will get the capital of 
    capital: the capital of the country stated
}
]
"""
messages = [{'text': system_prompt}]  
response = get_chatbot_response(messages)
print(response)

```json
[
  {
    "country": "France",
    "capital": "Paris"
  }
]
```



### 2) structured input

In [11]:
system_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'text': system_prompt}]  
response = get_chatbot_response(messages)
print(response)

1. Italy - Rome
2. France - Paris
3. Germany - Berlin



### 3)  Chain of thought : Give model time to think

In [12]:
system_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'text': system_prompt}]  
response = get_chatbot_response(messages)
print(response)


```json
{
  "steps": "Following BEDMAS (Brackets, Exponents, Division and Multiplication from left to right, Addition and Subtraction from left to right):\n\n1. Division: 259 / 2 = 129.5\n2. Multiplication (1): 129.5 * 8654 = 1120000 + 5190 + 43270 + 7740 + 1295 = 1122363\n3. Multiplication (2): 91072 * 33 = 2997376 \n4. Addition: 1122363 + 2997376 = 4119739\n5. Subtraction: 4119739 - 12971 = 4106768",
  "result": 4106768
}
```



###  RAG - Retrieval Augmented Generation

In [13]:
embedding_model_name = os.getenv("EMBEDDING_MODEL_NAME", "models/text-embedding-004")

In [14]:
def get_embedding(text_input):
    result = genai.embed_content(
        model=embedding_model_name , 
        content=text_input
    )

    embedding = result['embedding']
    return embedding

In [15]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [16]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [17]:
data = [iphone_16,samsung_s23]

In [18]:
user_prompt = """What's new in iphone 16?"""
user_prompt_embeddings = get_embedding(system_prompt)
print(user_prompt_embeddings)

[0.0061438465, -0.001949737, -0.014181823, 0.009104185, 0.03299688, 0.06654366, 0.06271397, 0.05014754, -0.0008453271, 0.0136248255, -0.015688933, -0.0049665333, 0.023984237, 0.0128566725, 0.061009087, -0.061809275, 0.037997022, 0.03829026, -0.10877881, 0.0024127471, 0.027050512, -0.027518885, 0.0044580325, 0.01585691, -0.063779265, -0.010496415, -0.03348543, -0.0576879, 0.0561909, 0.009032659, 0.036307517, 0.008656795, -0.047088824, -0.06696679, 0.019250266, 0.03151103, -0.008380219, 0.022357618, 0.044863444, -0.013353064, -0.06429306, -0.01801903, 0.0059479866, 0.0168679, -0.017955363, -0.03773597, -0.010616519, -0.017767087, -0.020344868, 0.044458754, 0.050184574, -0.020537598, -0.0040852334, -0.040704187, -0.024916304, -0.0027411678, -0.04946308, -0.039144795, 0.032749783, -0.014032241, -0.033588644, -0.06989241, -0.012718956, -0.04035856, -0.014201617, 0.030976135, -0.038853675, -0.04340171, -0.005443529, 0.033408187, 0.013368484, 0.027578896, -0.040732045, -0.009081936, 0.0289154

In [19]:
data_embeddings = [get_embedding(x) for x in data]
data_embeddings_reshaped = np.array(data_embeddings) 

In [20]:
user_prompt_embeddings_reshaped = np.array(user_prompt_embeddings).reshape(1, -1) 

In [21]:
data_similarity_scores = cosine_similarity(user_prompt_embeddings_reshaped, data_embeddings_reshaped)

closest_entry_index = data_similarity_scores.argmax()

closest_entry = data[closest_entry_index]
closest_entry

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [22]:
user_prompt_with_data = f"""
{closest_entry}

{user_prompt}
"""

In [23]:
messages = [{'text': user_prompt_with_data}]  
response = get_chatbot_response(messages)
print(response)

The iPhone 16 boasts several key updates:

* **Larger Displays & Thinner Bezels:**  Larger screen sizes across all models (6.1-inch and 6.7-inch for base models, 6.3-inch and 6.9-inch for Pro models) with the thinnest bezels yet on an Apple product.  The screens also utilize a more durable Ceramic Shield.

* **Improved Performance:** The A18 chip (A18 Pro for Pro models) offers significant performance boosts, including enhanced neural engine, faster GPU, and improved machine learning capabilities.

* **Enhanced Camera Systems:**  The base model features a 48MP main sensor, while the Pro models include a 48MP Ultra Wide and a 5x telephoto camera, along with a new "Camera Control" button for improved photographic flexibility.

* **Advanced Audio Features:** "Audio Mix" uses machine learning to separate background noise from speech during video recording.

* **Extended Battery Life:**  Significantly improved battery life, especially in the iPhone 16 Pro Max, which boasts the longest batte